In [5]:
import torch
import torch.nn as nn
from torch.autograd import Variable


class ConvLSTMCell(nn.Module):
    def __init__(self, input_channels, hidden_channels, kernel_size):
        super(ConvLSTMCell, self).__init__()

        assert hidden_channels % 2 == 0

        self.input_channels = input_channels
        self.hidden_channels = hidden_channels
        self.kernel_size = kernel_size
        self.num_features = 4

        self.padding = int((kernel_size - 1) / 2)

        self.Wxi = nn.Conv2d(self.input_channels, self.hidden_channels, self.kernel_size, 1, self.padding, bias=True)
        self.Whi = nn.Conv2d(self.hidden_channels, self.hidden_channels, self.kernel_size, 1, self.padding, bias=False)
        self.Wxf = nn.Conv2d(self.input_channels, self.hidden_channels, self.kernel_size, 1, self.padding, bias=True)
        self.Whf = nn.Conv2d(self.hidden_channels, self.hidden_channels, self.kernel_size, 1, self.padding, bias=False)
        self.Wxc = nn.Conv2d(self.input_channels, self.hidden_channels, self.kernel_size, 1, self.padding, bias=True)
        self.Whc = nn.Conv2d(self.hidden_channels, self.hidden_channels, self.kernel_size, 1, self.padding, bias=False)
        self.Wxo = nn.Conv2d(self.input_channels, self.hidden_channels, self.kernel_size, 1, self.padding, bias=True)
        self.Who = nn.Conv2d(self.hidden_channels, self.hidden_channels, self.kernel_size, 1, self.padding, bias=False)

        self.Wci = None
        self.Wcf = None
        self.Wco = None

    def forward(self, x, h, c):
        ci = torch.sigmoid(self.Wxi(x) + self.Whi(h) + c * self.Wci)
        cf = torch.sigmoid(self.Wxf(x) + self.Whf(h) + c * self.Wcf)
        cc = cf * c + ci * torch.tanh(self.Wxc(x) + self.Whc(h))
        co = torch.sigmoid(self.Wxo(x) + self.Who(h) + cc * self.Wco)
        ch = co * torch.tanh(cc)
        return ch, cc

    def init_hidden(self, batch_size, hidden, shape):
        if self.Wci is None:
            self.Wci = nn.Parameter(torch.zeros(1, hidden, shape[0], shape[1]))
            self.Wcf = nn.Parameter(torch.zeros(1, hidden, shape[0], shape[1]))
            self.Wco = nn.Parameter(torch.zeros(1, hidden, shape[0], shape[1]))
        else:
            assert shape[0] == self.Wci.size()[2], 'Input Height Mismatched!'
            assert shape[1] == self.Wci.size()[3], 'Input Width Mismatched!'
        return (Variable(torch.zeros(batch_size, hidden, shape[0], shape[1])),
                Variable(torch.zeros(batch_size, hidden, shape[0], shape[1])))


class ConvLSTM(nn.Module):
    # input_channels corresponds to the first input feature map
    # hidden state is a list of succeeding lstm layers.
    def __init__(self, input_channels, hidden_channels, kernel_size, step=1, effective_step=[1]):
        super(ConvLSTM, self).__init__()
        self.input_channels = [input_channels] + hidden_channels
        self.hidden_channels = hidden_channels
        self.kernel_size = kernel_size
        self.num_layers = len(hidden_channels)
        self.step = step
        self.effective_step = effective_step
        self._all_layers = []
        for i in range(self.num_layers):
            name = 'cell{}'.format(i)
            cell = ConvLSTMCell(self.input_channels[i], self.hidden_channels[i], self.kernel_size)
            setattr(self, name, cell)
            self._all_layers.append(cell)

    def forward(self, input):
        internal_state = []
        outputs = []
        for step in range(self.step):
            x = input
            for i in range(self.num_layers):
                # all cells are initialized in the first step
                name = 'cell{}'.format(i)
                if step == 0:
                    bsize, _, height, width = x.size()
                    (h, c) = getattr(self, name).init_hidden(batch_size=bsize, hidden=self.hidden_channels[i],
                                                             shape=(height, width))
                    internal_state.append((h, c))

                # do forward
                (h, c) = internal_state[i]
                x, new_c = getattr(self, name)(x, h, c)
                internal_state[i] = (x, new_c)
            # only record effective steps
            if step in self.effective_step:
                outputs.append(x)

        return outputs, (x, new_c)

In [14]:
from torch.optim import Adam

convlstm = ConvLSTM(input_channels=512, hidden_channels=[128, 64, 64, 32, 32], kernel_size=3, step=5,
                    effective_step=[4])
loss_fn = torch.nn.MSELoss()
optimizer = Adam(convlstm.parameters(), lr=0.001)

input = Variable(torch.randn(1, 512, 64, 32))
target = Variable(torch.randn(1, 32, 64, 32)).double()

# output = convlstm(input)
# output = output[0][0].double()
# res = torch.autograd.gradcheck(loss_fn, (output, target), eps=1e-6, raise_exception=True)
# print(res)

num_epochs = 100
for epoch in range(num_epochs):
    optimizer.zero_grad()
    output = convlstm(input)
    output = output[0][0].double()
    loss = loss_fn(output, target)
    loss.backward()
    optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')


Epoch [1/100], Loss: 1.0016895097436098
Epoch [2/100], Loss: 1.0006560176339039
Epoch [3/100], Loss: 0.9995398638365083
Epoch [4/100], Loss: 0.9981229490558847
Epoch [5/100], Loss: 0.9961686438159006
Epoch [6/100], Loss: 0.9932281265933663
Epoch [7/100], Loss: 0.988249847070243
Epoch [8/100], Loss: 0.9806718405743011
Epoch [9/100], Loss: 0.9678037742029018
Epoch [10/100], Loss: 0.942053006647846
Epoch [11/100], Loss: 0.9251712045825032
Epoch [12/100], Loss: 0.8927080727878838
Epoch [13/100], Loss: 0.8767168940144288
Epoch [14/100], Loss: 0.8534190366007661
Epoch [15/100], Loss: 0.8304248631768127
Epoch [16/100], Loss: 0.8186437499748442
Epoch [17/100], Loss: 0.7928802890547193
Epoch [18/100], Loss: 0.7720165659785115
Epoch [19/100], Loss: 0.7567803676172136
Epoch [20/100], Loss: 0.7361922555798844
Epoch [21/100], Loss: 0.7192944833019449
Epoch [22/100], Loss: 0.7023807788098542
Epoch [23/100], Loss: 0.6858882748021875
Epoch [24/100], Loss: 0.6697321420889324
Epoch [25/100], Loss: 0.655

torch.Size([1, 32, 64, 32])